In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import re
from collections import defaultdict
from pprint import pprint
import re
import urllib
import urllib.parse
import time

from fake_useragent import UserAgent
ua = UserAgent()
ua.random

%matplotlib inline

## Helper function for getting lyrics from songlyrics.com and lyricsmode.com

In [3]:
#Getting Lyrics Helper Function

import urllib
import urllib.parse

def songlyrics(artist, title):
    
    title = title.replace('?', '')
    title = title.replace('’', '-')
    artist = artist.replace('-', '_')
    artist = artist.replace('’', '-')
    
    if title == 'Beach Baby':
        return None
    
    headers = { 'User-Agent': ua.random }

    try:
        url = ('http://www.songlyrics.com/%s/%s-lyrics/' % (artist,title))
    except:
        return None
    
    response = requests.get(url, headers=headers)
    page = response.text
    lyric_soup = BeautifulSoup(page)
    
    lyrics = lyric_soup.findAll(attrs= {'id' : 'songLyricsDiv'})
    if not lyrics:
        return None
    else:
        if str(lyrics[0]).startswith("<p class=\"songLyricsV14 iComment-text\" id=\"songLyricsDiv\">Sorry, we have no"):
            print (lyrics[0])
            return None
        try:
            return re.sub('<[^<]+?>', '', ''.join(str(lyrics[0])))
        except:
            return None


def lyricsmode(artist, title):
    
    title = title.replace('?', '')
    title = title.replace('’', '')
    artist = artist.replace('’', '')

    headers = { 'User-Agent': ua.random }    
    
    try:
        url = 'http://www.lyricsmode.com/lyrics/%s/%s/%s.html' % (artist[0],artist, title)
    except:
        return ('Error connecting')
    
    response = requests.get(url, headers=headers)
    page = response.text
    lyric_soup = BeautifulSoup(page)
    
    #lyricsmode places the lyrics in a span with an id of "lyrics"
    lyrics = lyric_soup.findAll(attrs= {'id' : 'lyrics_text'})
    if not lyrics:
        return ('Lyrics not found.')
    try:
        return re.sub('<[^<]+?>', '', ''.join(str(lyrics[0])))
    except:
        return ('Sorry, an error occurred while parsing the lyrics.')



def get_lyrics(artist, title):
    lyr = songlyrics(artist, title)
    if not lyr:
        lyr = lyricsmode(artist, title)
    return lyr

In [ ]:
get_lyrics('Justin Bieber', "what do you mean")

## Actually Scraping

In [ ]:
#Looking through years

years = list(range(1991,2016))

for year in years:
    print('Getting year ' + str(year))
    
    #Finding top 100 songs per year
    url = 'http://billboardtop100of.com/' + str(year) + '-2/'
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page)
    
    counter = 1
    rank = []
    artist = []
    song = []

    for element in soup.find_all("td"):
        if counter == 1:
            rank.append(element.text)
            counter += 1

        elif counter == 2:
            artist.append(element.text)
            counter += 1

        elif counter == 3:
            song.append(element.text)
            counter = 1
            
    df = pd.DataFrame(columns = ['Artist', 'Title', 'Year'])

    df.Artist = artist
    df.Title = song
    df.Year = year
    
    clean_names = []
    for name in df['Artist']:
        if 'feat' in name:
            name = name.split(' feat.', 1)[0]
        if 'and' in name:
            name = name.split(' and', 1)[0]
        if '$' in name:
            name = name.replace('$', 's')
        if '.' in name:
            name = name.replace('.', '')
        clean_names.append(name)

    df['Artist'] = clean_names
    
    
    #Scrape the song lyrics
    from fake_useragent import UserAgent
    ua = UserAgent()
    ua.random
    
    lyrics = []

    for index, row in df.iterrows():
        try:
            print('     Scraping ' + row['Artist'] + ' - ' + row['Title'])
            scraped = get_lyrics(row['Artist'], row['Title'])
            lyrics.append(scraped)
            print('     Done')
            time.sleep(1)
        except:
            lyrics.append('Lyrics not found.')
    
    df['Lyrics'] = lyrics
    
    df.to_csv(str(year) + 'songs.csv')
    print('Done with ' + str(year))
    print()

## Scraping 2013

In [4]:
url = 'http://billboardtop100of.com/2013-2/'
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page)
pprint (soup)

<!DOCTYPE html>
<!--[if IE 7]>
<html class="ie ie7" lang="en-US" prefix="og: http://ogp.me/ns#">
<![endif]--><!--[if IE 8]>
<html class="ie ie8" lang="en-US" prefix="og: http://ogp.me/ns#">
<![endif]--><!--[if !(IE 7) | !(IE 8)  ]><!--><html lang="en-US" prefix="og: http://ogp.me/ns#">
<!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="IE=9; IE=8; IE=7; IE=EDGE" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="http://gmpg.org/xfn/11" rel="profile"/>
<link href="http://billboardtop100of.com/wp/xmlrpc.php" rel="pingback"/>
<!-- html5shiv for IE8 and less  -->
<!--[if lt IE 9]>
				      <script src="http://billboardtop100of.com/wp/wp-content/themes/customizr/inc/assets/js/html5.js"></script>
				    <![endif]-->
<title>2013 - Billboard Top 100</title>
<!-- This site is optimized with the Yoast SEO plugin v3.2.4 - https://yoast.com/wordpress/plugins/seo/ -->
<link href="http://billboardtop100of.com/2013-2/" rel=

/Users/justinchien/anaconda/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [9]:
artist = []
song = []

for element in soup.find("small"):

    if 'Mike WiLL Made-It Featuring Miley Cyrus, Wiz Khalifa & Juicy J' in element:
        artist.append(element)
        song.append('23')
    else:
        try:
            artist.append(element.split(' – ', 1)[0].split('\xa0',1)[1].strip())
            song.append(element.split(' – ',1)[1].strip())
        except:
            continue

song.append('Beware')

df = pd.DataFrame(columns = ['Artist', 'Title', 'Year'])

df.Artist = artist
df.Title = song
df.Year = 2013

clean_names = []
for name in df['Artist']:
    if 'Featuring' in name:
        name = name.split(' Featuring', 1)[0]
    if 'Feat' in name:
        name = name.split(' Feat', 1)[0]
    if 'with' in name:
        name = name.split(' with', 1)[0]
    if 'With' in name:
        name = name.split(' With', 1)[0]
    if '&' in name:
        name = name.split(' &', 1)[0]
    if '+' in name:
        name = name.split(' +', 1)[0]
    if '$' in name:
        name = name.replace('$', 's')
    if '.' in name:
        name = name.replace('.', ' ')
    if name == 'Weeknd':
        name = 'The Weeknd'
    if name == 'P!nk':
        name = 'Pink'
        
    clean_names.append(name)

df['Artist'] = clean_names


clean_titles = []
for title in df['Title']:
    if '?' in title:
        title = title.replace('?', '')
    if '/' in title:
        title = title.replace('/', '')    
    if '&' in title:
        title = title.replace(' & ', ' ')
    if '(' in title:
        title = title.replace('(', '')
    if ')' in title:
        title = title.replace(')', '')
    if '\'' in title:
        title = title.replace('\'', '')
        
        
    if 'Cups Pitch Perfect' in title:
        title = "Cups Pitch Perfects When Im gone"
    if title == '#thatPOWER':
        title = 'That Power'
    if title == '#Beautiful':
        title = 'Beautiful'
        
    clean_titles.append(title)
    
df['Title'] = clean_titles



#Scrape the song lyrics
from fake_useragent import UserAgent
ua = UserAgent()
ua.random

lyrics = []

for index, row in df.iterrows():
    try:
        print('     Scraping ' + row['Artist'] + ' - ' + row['Title'])
        scraped = get_lyrics(row['Artist'], row['Title'])
        lyrics.append(scraped.strip())
        print('     Done')
        time.sleep(1)
    except:
        lyrics.append('Lyrics not found.')

df['Lyrics'] = lyrics

df.to_csv('2013songs.csv')

     Scraping Macklemore - Can’t Hold Us


/Users/justinchien/anaconda/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


     Done
     Scraping Baauer - Harlem Shake
     Done
     Scraping Pink - Just Give Me A Reason
     Done
     Scraping Robin Thicke - Blurred Lines
     Done
     Scraping Lorde - Royals
     Done
     Scraping Robin Thicke - Blurred Lines
     Done
     Scraping Katy Perry - Roar
     Done
     Scraping Justin Timberlake - Mirrors
     Done
     Scraping Daft Punk - Get Lucky
     Done
     Scraping Miley Cyrus - We Can’t Stop
     Done
     Scraping Miley Cyrus - Wrecking Ball
     Done
     Scraping will i am - Scream Shout
     Done
     Scraping Justin Timberlake - Suit Tie
     Done
     Scraping Rihanna - Stay
     Done
     Scraping Eminem - Berzerk
     Done
     Scraping Eminem - The Monster
     Done
     Scraping Drake - Hold On, We’re Going Home
     Done
     Scraping Florida Georgia Line - Cruise
     Done
     Scraping Bruno Mars - Treasure
     Done
     Scraping Avicii - Wake Me Up!
     Done
     Scraping Jay Z - Holy Grail
     Done
     Scraping Drake - Started

In [10]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.precision', 3)
df

,Artist,Title,Year,Lyrics
0,Macklemore,Can’t Hold Us,2013,"[Intro:]\r\nAy, ay, ay\r\nGood to see you, com..."
1,Baauer,Harlem Shake,2013,Con los teroristas\n(Do the Harlem Shake)\n\nC...
2,Pink,Just Give Me A Reason,2013,Right from the start\r\nYou were a thief\r\nYo...
3,Robin Thicke,Blurred Lines,2013,We do not have the lyrics for Blurred Lines (B...
4,Lorde,Royals,2013,[Verse 1]\nI've never seen a diamond in the fl...
5,Robin Thicke,Blurred Lines,2013,We do not have the lyrics for Blurred Lines (B...
6,Katy Perry,Roar,2013,I used to bite my tongue and hold my breath\r\...
7,Justin Timberlake,Mirrors,2013,[Verse 1]\r\nAren't you somethin' to admire\r\...
8,Daft Punk,Get Lucky,2013,[Verse 1: Pharrell]\r\nLike the legend of the ...
9,Miley Cyrus,We Can’t Stop,2013,It's our party we can do what we want (no dram...


## Scraping 2015

In [ ]:
url = 'http://billboardtop100of.com/2015-2/'
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page)

counter = 1
rank = []
artist = []
song = []

for element in soup.find_all("td"):
    if counter == 1:
        rank.append(element.text.strip())
        counter += 1
        
    elif counter == 2:
        artist.append(element.text.strip())
        counter += 1
        
    elif counter == 3:
        dirty = element.text.strip()
        if 'LYRICS' in dirty:
            clean = dirty.split('\nLYRICS', 1)[0]
            song.append(clean)
        else:
            song.append(element.text.strip())
        counter = 1
        
df = pd.DataFrame(columns = ['Artist', 'Title', 'Year'])

df.Artist = artist
df.Title = song
df.Year = 2015

clean_names = []
for name in df['Artist']:
    if 'feat' in name:
        name = name.split(' feat.', 1)[0]
    if 'and' in name:
        name = name.split(' and', 1)[0]
    if '$' in name:
        name = name.replace('$', 's')
    if '.' in name:
        name = name.replace('.', '')
    if name == 'Weeknd':
        name = 'The Weeknd'
        
    clean_names.append(name)

df['Artist'] = clean_names


clean_titles = []
for title in df['Title']:
    if '?' in title:
        title = title.replace('?', '')
    if '/' in title:
        title = title.replace('/', '')
    
    if 'B****' in title:
        title = 'Bitch better have my money'        
    if 'Earned It (Fifty Shades' in title:
        title = 'Earned It'
    if 'I’m Not The Only\nOne' in title:
        title = "I'm not the only one"
    if 'What Do Yo' in title:
        title = 'What do you mean'
        
    clean_titles.append(title)
    
df['Title'] = clean_titles
        
#Scrape the song lyrics
from fake_useragent import UserAgent
ua = UserAgent()
ua.random

lyrics = []

for index, row in df.iterrows():
    try:
        print('     Scraping ' + row['Artist'] + ' - ' + row['Title'])
        scraped = get_lyrics(row['Artist'], row['Title'])
        lyrics.append(scraped.strip())
        print('     Done')
        time.sleep(1)
    except:
        lyrics.append('Lyrics not found.')

df['Lyrics'] = lyrics

df.to_csv('2015songs.csv')

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.precision', 3)
df

# Exploration

In [ ]:
url = 'http://billboardtop100of.com/1956-2/'
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page)


In [ ]:
counter = 1
rank = []
artist = []
song = []

for element in soup.find_all("td"):
    if counter == 1:
        rank.append(element.text)
        counter += 1
        
    elif counter == 2:
        artist.append(element.text)
        counter += 1
        
    elif counter == 3:
        song.append(element.text)
        counter = 1


In [ ]:
df = pd.DataFrame(columns = ['Artist', 'Title'])

In [ ]:
df.Artist = artist
df.Title = song

In [ ]:
clean_names=[]

for name in df['Artist']:
    
    if 'feat' in name:
        name = name.split(' feat.', 1)[0]
    
    if 'and' in name:
        name = name.split(' and', 1)[0]
    
    if '$' in name:
        name = name.replace('$', 's')
        
    if '.' in name:
        name = name.replace('.', '')
        
    clean_names.append(name)
        
df['Artist'] = clean_names

In [ ]:
df['Title'][1]

## Scraping A to Z Lyrics

In [ ]:
from fake_useragent import UserAgent
ua = UserAgent()
ua.random

### LyricsMode

In [ ]:
import urllib
import urllib.parse

def songlyrics(artist, title):
    
    headers = { 'User-Agent': ua.random }

    try:
        url = ('http://www.songlyrics.com/%s/%s-lyrics/' % (artist,title))
    except:
        return None
    
    response = requests.get(url, headers=headers)
    page = response.text
    lyric_soup = BeautifulSoup(page)
    
    lyrics = lyric_soup.findAll(attrs= {'id' : 'songLyricsDiv'})
    
    if not lyrics:
        return None
    else:
        if str(lyrics[0]).startswith("<p class=\"songLyricsV14 iComment-text\" id=\"songLyricsDiv\">Sorry, we have no"):
            return None
        try:
            return re.sub('<[^<]+?>', '', ''.join(str(lyrics[0])))
        except:
            return None


def lyricsmode(artist, title):

    headers = { 'User-Agent': ua.random }    
    
    try:
        url = 'http://www.lyricsmode.com/lyrics/%s/%s/%s.html' % (artist[0],artist, title)
    except:
        return ('Error connecting')
    
    response = requests.get(url, headers=headers)
    page = response.text
    lyric_soup = BeautifulSoup(page)
    
    #lyricsmode places the lyrics in a span with an id of "lyrics"
    lyrics = lyric_soup.findAll(attrs= {'id' : 'lyrics_text'})
    if not lyrics:
        return ('Lyrics not found.')
    try:
        return re.sub('<[^<]+?>', '', ''.join(str(lyrics[0])))
    except:
        return ('Sorry, an error occurred while parsing the lyrics.')



def get_lyrics(artist, title):
    lyr = songlyrics(artist, title)
    if not lyr:
        lyr = lyricsmode(artist, title)
    return lyr

In [ ]:
#Testing
get_lyrics('Perry Como', 'Hot Diggity')

In [ ]:
import time
lyrics = []

for index, row in df.iterrows():
    print('Scraping ' + row['Artist'] + ' - ' + row['Title'])
    scraped = get_lyrics(row['Artist'], row['Title'])
    lyrics.append(scraped)
    print('Done')
    time.sleep(1)

In [ ]:
df['Lyrics'] = lyrics

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.precision', 3)
df

In [ ]:
get_lyrics('Joni James', 'have you heard?')

In [ ]:
songlyrics('Adele', 'Hellos')